In [50]:
#General import
from sklearn import metrics
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#Qiskit import
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.visualization import circuit_drawer
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap, ZFeatureMap
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import XGate, SXGate, RZGate, CXGate

#klearn import
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets


In [51]:
class Individual():
    
    def __init__(
        self,
        numberOfGenes: int,
        numberOfFeatures: int,
        numberOfQubits: int
    ) -> None:
    
        self._numberOfFeatures = numberOfFeatures
        self._numberOfGenes = numberOfGenes
        self._numberOfQubits = numberOfQubits
        self._genotype =  [[None, None, None, None] for _ in range(numberOfGenes)]
        
        
        
    def generateFirstIndividual(
        self
    ) -> None:
        
        for gene in range(self._numberOfGenes):
            
            gate = random.choice([XGate, SXGate, RZGate, CXGate])
            
            self._genotype[gene][0] = gate
            
            if gate == RZGate:
                
                qubit = random.randint(0, self._numberOfQubits-1)
                feature = random.randint(0, self._numberOfFeatures-1)
                self._genotype[gene][1] = qubit
                self._genotype[gene][2] = None
                self._genotype[gene][3] = feature
                
            elif gate == CXGate:
                
                control_qubit = random.randint(0, self._numberOfQubits-1)
                target_qubit = random.randint(0, self._numberOfQubits-1)
                while target_qubit == control_qubit:
                    target_qubit = random.randint(0, self._numberOfQubits-1)
                    
                self._genotype[gene][1] = target_qubit
                self._genotype[gene][2] = control_qubit
                self._genotype[gene][3] = None
                
            else:
                
                qubit = random.randint(0, self._numberOfQubits-1)
                self._genotype[gene][1] = qubit
                self._genotype[gene][2] = None
                self._genotype[gene][3] = None
                
                
    def getFenotype(
        self
    ) -> QuantumCircuit:
        
        trainingParams = ParameterVector('x', self._numberOfFeatures)
        
        fenotype = QuantumCircuit(self._numberOfQubits)
        
        for gene in range(self._numberOfGenes):
                
                gate = self._genotype[gene][0]
                
                if gate == RZGate:
                    
                    fenotype.append(gate(trainingParams[self._genotype[gene][3]]), [self._genotype[gene][1]])
                    
                elif gate == CXGate:
                        
                    fenotype.append(gate(), [self._genotype[gene][2],  self._genotype[gene][1] ])
                    
                else:
                
                    fenotype.append(gate(), [self._genotype[gene][1] ])        

        return fenotype

        

                

            
            
        

    

            


In [61]:
individual = Individual (numberOfFeatures=2, numberOfGenes=4, numberOfQubits=2)
individual.generateFirstIndividual()
qc = individual.getFenotype()
print(circuit_drawer(qc))


     ┌───┐┌──────────┐     
q_0: ┤ X ├┤ Rz(x[1]) ├─────
     └─┬─┘└──┬───┬───┘┌───┐
q_1: ──■─────┤ X ├────┤ X ├
             └───┘    └───┘
